# Generate near-boundary points

## Introduction

An estimation about basin boundary is good if it classifies points near the boundary well. Therefore, to obtain a dataset which consists of many near-boundary points is desirable for us to subsequently train our model. We adapt the following approach: Consider an attractor of interest from the given dynamical system. We start from uniformly sampled points from the region of interest, then we separate the sampled points into two(+ and -) classes, depending on whether the trajectory starting from the point will tend to the attractor. We consider all pairs of data points, where for each pair one point is taken from the positive class and the other is taken from the negative class. For such a pair, since they belong to different classes, the actual boundary must go in between them. We perform bisection to narrow down the margin of estimation until the margin becomes smaller than $\delta$ ($\delta$ is a hyperparameter to control precision of estimation). We then save those points close to the boundary as a dataset.

## Lorenz system and parameters

We model the Lorenz system, which consists of the following three ODEs: 

$$x'=\sigma(y-x) \\
y'=rx-y-xz \\
z'=xy-\beta z$$

In the script, we set $\sigma=10, \beta = \frac{8}{3}, r=10$. With this set of parameters, we have that $(\sqrt{24}, \sqrt{24}, 9)$ is a stable equilibrium. We therefore obtain a dataset by simulating the trajectory of 500 sample points. If the trajectory is attracted, we label it as +1, otherwise -1. We have obtained an initial dataset of 100,000 points by sampling from $(x_{0},y_{0},z_{0}) \in (-50,50)\times (-50,50) \times (-50,50)$ uniformly at random and compute their trajectories.

In [1]:
import numpy as np
import math
import pandas as pd
from scipy.integrate import solve_ivp

In [2]:
## Load the dataset
df = pd.read_csv('dataset_large.csv')

In [3]:
## Split the dataset according to the class label
df_n1 = df[df['attracted'] == -1]
df_1 = df[df['attracted'] == 1]

In [4]:
## Lorenz system
def lorenz(t, X, sigma, beta, r):
    """The Lorenz equations."""
    x, y, z = X
    xp = sigma*(y - x)
    yp = r*x - y - x*z
    zp = -beta*z + x*y
    return xp, yp, zp

sigma, beta, r = 10, 8/3, 10

In [5]:
# Check if the trajectory is attracted to the concerned Lorenz attractor
def is_attracted(x, y, z):
    return (abs(x-math.sqrt(24))<0.01) and (abs(y-math.sqrt(24))<0.01) and (abs(z-9)<0.01)

In [6]:
## Implement the simulation process and decide if the trajectory is attracted by the Lorenz attractor
def simulation(x0 ,y0 ,z0):
    tmax, n = 1500, 100000
    soln = solve_ivp(lorenz, (0, tmax), (x0, y0, z0), args=(sigma, beta, r),dense_output=True)
    t = np.linspace(0, tmax, n)
    x, y, z = soln.sol(t)
    return is_attracted(x[n-1], y[n-1], z[n-1])

We define the bisection routine as follows: We accept two points $a$, $b$, and precision threshold $\delta$ as input. $a$ is from positive class and $b$ is from negative class. The algorithm terminates if $a$ and $b$ are of distance within $\delta$, that means the boundary is estimated by a margin of length $\leq \delta$, and then $a$ and $b$ are returned as near-boundary points. Otherwise, the algorithm takes midpoint $c$ and check which class $c$ belongs to, and go to one of the recursion cases depending on the result. By this approach, the margin is halved each time.

In [7]:
def bisection(a, b, delta):
    distance = np.linalg.norm(np.array([a[0]-b[0], a[1]-b[1], a[2]-b[2]]))
    if distance < delta:
        return (a, b)
    else:
        c = ((a[0]+b[0])/2, (a[1]+b[1])/2, (a[2]+b[2])/2)
        if simulation(c[0], c[1], c[2]):
            return bisection(c, b, delta)
        else:
            return bisection(a, c, delta)

In [8]:
## Test bisection
a = (-4.7239826783, 20.3245610659, -30.9716189993) ## label +1
b = (38.3553618552, 38.3553618552, -35.7140693026) ## label -1

a_near, b_near = bisection(a, b, 0.01)
print(simulation(a_near[0], a_near[1], a_near[2]))
print(simulation(b_near[0], b_near[1], b_near[2]))
print(a_near)
print(b_near)

True
False
(10.3790297118626, 26.645906264492478, -32.63425538492959)
(10.3842884209121, 26.64810728997945, -32.634834297320126)


In [12]:
# Initialize an empty list to store the rows
rows = []

# Get the total number of iterations
total_iterations = len(df_1) * len(df_n1)
current_iteration = 0

# Iterate over each possible pair to generate near-boundary points
for i, row_a in df_1.iterrows():
    for j, row_b in df_n1.iterrows():
        a = (df_1.iloc[i]['x0'], df_1.iloc[i]['y0'], df_1.iloc[i]['z0'])
        b = (df_n1.iloc[j]['x0'], df_n1.iloc[j]['y0'], df_n1.iloc[j]['z0'])
        a_near, b_near = bisection(a, b, 0.01)

        # Append a_near and b_near to the list as dictionaries
        rows.append({'x0': a_near[0], 'y0': a_near[1], 'z0': a_near[2], 'attracted': 1})
        rows.append({'x0': b_near[0], 'y0': b_near[1], 'z0': b_near[2], 'attracted': -1})

        # Update and print the progress every 10,000 points
        current_iteration += 1
        if current_iteration % 10000 == 0:
            print(f'Progress: {current_iteration}/{total_iterations}')

# Convert the list of dictionaries to a DataFrame
df_near = pd.DataFrame(rows)

KeyboardInterrupt: 